In [5]:
# We use the Hyperparameter Tuner
import sagemaker
from sagemaker.tuner import IntegerParameter
from sagemaker.sklearn.estimator import SKLearn
from sagemaker import get_execution_role

FRAMEWORK_VERSION = "0.23-1"

# Define exploration boundaries
hyperparameter_ranges = {
   "n-estimators": IntegerParameter(20, 100),
   "min-samples-leaf": IntegerParameter(2, 6),
}

sklearn_estimator = SKLearn(
   entry_point="script.py",
   role=get_execution_role(),
   instance_count=1,
   instance_type="ml.c5.xlarge",
   framework_version=FRAMEWORK_VERSION,
   base_job_name="rf-scikit",
   hyperparameters={
       "n-estimators": 100,
       "min-samples-leaf": 3,
   },
)
trainpath = "s3://my-ml-databucket/sagemaker/sklearncontainer/dry-bean-train.csv"
testpath = "s3://my-ml-databucket/sagemaker/sklearncontainer/dry-bean-test.csv"
# Launch training job, with asynchronous call
sklearn_estimator.fit({"train": trainpath, "test": testpath}, wait=True)

INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker:Creating training-job with name: rf-scikit-2025-06-15-02-41-16-331


2025-06-15 02:41:18 Starting - Starting the training job...
2025-06-15 02:41:51 Downloading - Downloading input data...
2025-06-15 02:42:06 Downloading - Downloading the training image...
2025-06-15 02:42:47 Training - Training image download completed. Training in progress.
2025-06-15 02:42:47 Uploading - Uploading generated training model2025-06-15 02:42:42,515 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2025-06-15 02:42:42,518 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2025-06-15 02:42:42,555 sagemaker_sklearn_container.training INFO     Invoking user training script.
2025-06-15 02:42:42,725 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2025-06-15 02:42:42,736 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2025-06-15 02:42:42,746 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2025-06-15 02:42:42

In [6]:
# Create Tuner object
# Create Optimizer
Optimizer = sagemaker.tuner.HyperparameterTuner(
   estimator=sklearn_estimator,
   hyperparameter_ranges=hyperparameter_ranges,
   base_tuning_job_name="RF-tuner",
   objective_type="Maximize",
   objective_metric_name="balanced-accuracy",
   metric_definitions=[
       {"Name": "balanced-accuracy", "Regex": "Test balanced accuracy: ([0-9.]+).*$"}
   ],  # Extract tracked metric from logs with regexp
   max_jobs=10,
   max_parallel_jobs=2,
)

Optimizer.fit({"train": trainpath, "test": testpath})

# Get tuner results in a df
results = Optimizer.analytics().dataframe()

while results.empty:
   time.sleep(1)
   results = Optimizer.analytics().dataframe()

results.head()

# retrieve the best estimator
best_estimator = Optimizer.best_estimator()

INFO:sagemaker:Creating hyperparameter tuning job with name: RF-tuner-250615-0243


..............................................................!

2025-06-15 02:48:39 Starting - Found matching resource for reuse
2025-06-15 02:48:39 Downloading - Downloading the training image
2025-06-15 02:48:39 Training - Training image download completed. Training in progress.
2025-06-15 02:48:39 Uploading - Uploading generated training model
2025-06-15 02:48:39 Completed - Resource retained for reuse


In [7]:
# *********************Deploying Models to End Point *********************

In [8]:
import boto3
sm_boto3 = boto3.client("sagemaker")
sess = sagemaker.Session()
region = sess.boto_session.region_name
BUCKET_URI = "s3://my-ml-databucket"
BUCKET_NAME = "my-ml-databucket"
DATASET_PATH = f"{BUCKET_URI}/dry_bean.csv"
TARGET_NAME = "Class"

artifact_path = sm_boto3.describe_training_job(
   TrainingJobName=best_estimator.latest_training_job.name
)["ModelArtifacts"]["S3ModelArtifacts"]

print("Model artifact persisted at " + artifact_path)

Model artifact persisted at s3://sagemaker-us-east-1-586794478771/RF-tuner-250615-0243-009-6e87c474/output/model.tar.gz


In [ ]:
# Deploy Model 
from sagemaker.sklearn.model import SKLearnModel

model = SKLearnModel(
   model_data=artifact_path,
   role=get_execution_role(),
   entry_point="script.py",
   framework_version=FRAMEWORK_VERSION,
)

predictor = model.deploy(instance_type="ml.c5.large", initial_instance_count=1)

sm_boto3.delete_endpoint(EndpointName=predictor.endpoint)

INFO:sagemaker:Creating model with name: sagemaker-scikit-learn-2025-06-15-02-49-03-468
INFO:sagemaker:Creating endpoint-config with name sagemaker-scikit-learn-2025-06-15-02-49-04-157
INFO:sagemaker:Creating endpoint with name sagemaker-scikit-learn-2025-06-15-02-49-04-157


----